## Ancestry Seach 

<b>Goal: </b>Multiple different spellings of a name can be referring to the same identity. We will use a phonetics library and Ancestry to fix this. An example: ```David Schaffer``` and ```David Schafer``` from `MA`. 

<b>Steps: </b>
1. Login to Emory's Ancestry subscription 
2. Iterate through ```agg_debt```, through each debt entry. 
3. Use a combination of phonetics fuzzy string matching and normal fuzzy string matching to determine if two names from a state are similar.  
4. Search each name in Ancestry: Edit URL (state and person's name). 
5. Check if there are any results for both person's name:
    - Yes: Check if one spelling of the name appears for both individuals (that's most likely the correct spelling of that name) 
    - No: Leave entries as two separate entries. 
6. Record name change in ```fixes``` list (save ```fixes``` as ```out.csv``` too). 
7. Run ```agg_debt``` through ```fixes```, making changes as necessary. 
8. Save ```agg_debt``` as a new .csv file.

<b style="color: red;">Note: Runtime is long. This is due to the fact there are over 200,000 debt entries and accessing Ancestry takes time too. </b>

In [1]:
# import necessary fuzzy string libraries 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.expected_conditions import element_to_be_clickable, presence_of_element_located
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from phonetics import metaphone
from rapidfuzz import fuzz
from joblib import Parallel, delayed, cpu_count
from itertools import zip_longest
import time 
import getpass
import pickle
import pandas as pd
import numpy as np

In [2]:
agg_debt = pd.read_csv('data/agg_debt_grouped.csv')

C:\Users\david\AppData\Local\Temp\ipykernel_1200\953087625.py:1: DtypeWarning: Columns (4,12,13,15,17,18,22,23,24,27,34) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_debt = pd.read_csv('data/agg_debt_grouped.csv')


In [3]:
agg_debt['to whom due | first name'] = agg_debt['to whom due | first name'].astype(str)
agg_debt['to whom due | last name'] = agg_debt['to whom due | last name'].astype(str)

In [4]:
name_changes = pd.read_csv('data/name_changes_david.csv')

In [5]:
agg_debt.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
0,0,0,0,NaN,NaN,NaN,NaN,NaN,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,0.0,C,8.0,27.0,1783.0,Elizabeth,Lowell,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2,2.0,G,8.0,27.0,1783.0,Joshua,Brackett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,3,3.0,C,9.0,2.0,1783.0,Phillips,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,4,4.0,B,9.0,2.0,1783.0,William,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# check to make sure pierce certificate first name and last name columns are properly swapped
agg_debt.loc[agg_debt['org_file'] == 'Pierce_Certs_cleaned_2019.xlsx'].head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
14015,14015,14015,14015,15296.0,NaN,NaN,NaN,NaN,Mason,Abbe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14016,14016,14016,14016,15298.0,NaN,NaN,NaN,NaN,John,Abbee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14017,14017,14017,14017,15299.0,NaN,NaN,NaN,NaN,Richard,Abbet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14018,14018,14018,14018,15300.0,NaN,NaN,NaN,NaN,Reuben,Abbett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14019,14019,14019,14019,15301.0,NaN,NaN,NaN,NaN,Richard,Abbett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
name_changes.tail()

,Unnamed: 0,title_org,title_new,first_name_org,last_name_org,first_name_new,last_name_new,cleaning case,file_loc,org_index
1627,1627,NaN,NaN,Jn,Gallaher Esq,John,Gallaher Esq,5,loan_office_certificates_9_states.xlsx,62583
1628,1628,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69947
1629,1629,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69948
1630,1630,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69949
1631,1631,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69950


In [8]:
# options
options = Options()
options.add_argument('--headless')
options.add_argument("--window-size=1000,1000")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--no-sandbox')   

In [9]:
agg_debt.state.unique()

array([nan, 'nh', 'nj', 'ny', 'ma', 'de', 'ct', 'va', 'pa', 'md', 'nc',
       'cs', 'ga', 'ri', 'f'], dtype=object)

In [10]:
# voter records and censuses available for every state 
records = {
    'nh':['https://www.ancestrylibrary.com/search/collections/5058/'],
    'nj':['https://www.ancestrylibrary.com/search/collections/2234/', 
          'https://www.ancestrylibrary.com/search/collections/3562/'],
    'ny':['https://www.ancestrylibrary.com/search/collections/5058/'],
    'ma':['https://www.ancestrylibrary.com/search/collections/5058/'], 
    'ct':['https://www.ancestrylibrary.com/search/collections/5058/'], 
    'va':['https://www.ancestrylibrary.com/search/collections/2234/', 
         'https://www.ancestrylibrary.com/search/collections/3578/'], 
    'pa':['https://www.ancestrylibrary.com/search/collections/2702/',
         'https://www.ancestrylibrary.com/search/collections/2234/',
         'https://www.ancestrylibrary.com/search/collections/3570/'],
    'md':['https://www.ancestrylibrary.com/search/collections/3552/'],
    'nc':['https://www.ancestrylibrary.com/search/collections/3005/', 
         'https://www.ancestrylibrary.com/search/collections/2234/'],
    'ga':['https://www.ancestrylibrary.com/search/collections/2234/'],
    'ri':['https://www.ancestrylibrary.com/search/collections/3571/']
}

# ancestry has unique urls for each state
residence_urls = {
    'nh':'_new+hampshire-usa_32',
    'nj':'_new+jersey-usa_33', 
    'ny':'_new+york-usa_35',
    'ma':'_massachusetts-usa_24',
    'ct':'_connecticut-usa_9',
    'va':'_virginia-usa_49', 
    'pa':'_pennsylvania-usa_41',
    'md':'_maryland-usa_23',
    'nc':'_north+carolina-usa_36',
    'ga':'_georgia-usa_13',
    'ri':'_rhode+island-usa_42'
}

In [11]:
%store -r ancestry_name_changes
%store -r fixes #record name necessary name changes here
%store -r checked0 #multiple debt entries for the same person: don't search these names again
%store -r checked1 #multiple debt entries for the same person: don't search these names again when comparing
%store -r rerun_rows

no stored variable or alias #record
no stored variable or alias name
no stored variable or alias necessary
no stored variable or alias name
no stored variable or alias changes
no stored variable or alias here
no stored variable or alias #multiple
no stored variable or alias debt
no stored variable or alias entries
no stored variable or alias for
no stored variable or alias the
no stored variable or alias same
no stored variable or alias person:
no stored variable or alias don't
no stored variable or alias search
no stored variable or alias these
no stored variable or alias names
no stored variable or alias again
no stored variable or alias #multiple
no stored variable or alias debt
no stored variable or alias entries
no stored variable or alias for
no stored variable or alias the
no stored variable or alias same
no stored variable or alias person:
no stored variable or alias don't
no stored variable or alias search
no stored variable or alias these
no stored variable or alias names
no 

In [12]:
print(len(ancestry_name_changes))

2041


In [13]:
# remove 'cs' (congress) and 'f' (foreign officers); these are not state, but specific regiments / types of officers
agg_debt_copy = agg_debt[(agg_debt['state'] != 'cs') & (agg_debt['state'] != 'f') & (agg_debt['state'] != 'de')]

# split dataframe based on state; makes searching faster
agg_debt_sp = agg_debt_copy.groupby('state')
agg_debts_st = [agg_debt_sp.get_group(x) for x in agg_debt_sp.groups]


for x in agg_debt_sp.groups:
    print(x)

ct
ga
ma
md
nc
nh
nj
ny
pa
ri
va


In [14]:
netid_xpath = '/html/body/div[1]/div[2]/section/div[1]/div/form/fieldset/div[1]/input'
password_xpath = '/html/body/div[1]/div[2]/section/div[1]/div/form/fieldset/div[2]/input'
login_btn0_xpath = '/html/body/main/div/div/div/a'
login_btn1_xpath = '/html/body/div[1]/div[2]/section/div[1]/div/form/fieldset/div[3]/button'

# ask for password and username 
username = input('username: ')
password = getpass.getpass(prompt='password: ')

driver_objs = {}
# create a new driver object for each state
for st in agg_debt_sp.groups:
    driver_objs[st] = [webdriver.Chrome(service=Service(ChromeDriverManager(version='114.0.5735.16').install()), options=options)]
# create a new wait object for each state
for st in agg_debt_sp.groups:
    webdriver_obj = driver_objs[st][0]
    driver_objs[st].append(WebDriverWait(webdriver_obj, 30))
# for each driver obj: access emory's ancestry's subscription 
for st in agg_debt_sp.groups:
    webdriver_obj = driver_objs[st][0]
    wait_obj = driver_objs[st][1]
    
    # go to emory's library 
    webdriver_obj.get('https://guides.libraries.emory.edu/ALE')
    wait_obj.until(element_to_be_clickable((By.XPATH, login_btn0_xpath))).click()
    
    # input login information and click 'login'
    netid_input = wait_obj.until(element_to_be_clickable((By.XPATH, netid_xpath)))
    netid_input.click()
    netid_input.send_keys(username)
    pass_input = wait_obj.until(element_to_be_clickable((By.XPATH, password_xpath)))
    pass_input.click()
    pass_input.send_keys(password) 
    wait_obj.until(element_to_be_clickable((By.XPATH, login_btn1_xpath))).click()
    
    webdriver_obj.get('https://www.ancestrylibrary.com/search/collections/5058/')
    
    print(webdriver_obj)
    

username: dcho52
password: ········


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 20.0MB/s]


<selenium.webdriver.chrome.webdriver.WebDriver (session="3cbbf4ec8f93ad0cc51cf02caa66533e")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="5c1e2539b619420c58e456c25b274c96")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="7fa0ab89712243fe95abb969d689e1a6")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="606ad0ac361a74cd0e71086d94f73fb5")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="b91eb85b9693a506bd738484ad180dd9")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="4b6e014e5299cdbd0df63bc7222d64fe")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="4add3d97efe11e35ee3d69cb0800be61")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="fbb75e9e04c6aa1c3e3847461f9454b5")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="f01b9f06a930a757f5e2552cc126cecc")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="080dacc719fe2a430998b7c9c496c5e8")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="04330c782a913

In [15]:
def ancestry_cleaning(agg_debt_st, state):
    # retrieve selenium chromedrivers associated with that state
    st_driver = driver_objs[state][0]
    wait_driver = driver_objs[state][1]
    # run ancestry search on agg debt file
    agg_debt_clean = compare_strings_vect(agg_debt_st.index, agg_debt_st['to whom due | title'], agg_debt_st['to whom due | first name'].astype(str), agg_debt_st['to whom due | last name'].astype(str), 
                                          agg_debt_st['org_index'], agg_debt_st['org_file'], state, st_driver, wait_driver) # using vectorization

In [16]:
%%capture
# loop through the state agg_debt one more time; compare row0 (original row) with all the other rows (row1)
def compare_strings(index, title0, fn0, ln0, org_index0, org_file0, state, st_driver, wait_driver):
    # make sure we haven't checked this name before (handles people who share the same fn & ln & live in same state) 
    # name0 = row0['to whom due | first name'] + ' ' + row0['to whom due | last name'] # uncomment when using apply
    agg_debt_st = agg_debt_sp.get_group(state)
    
    if (fn0, ln0, state) not in checked0:
        # compare both strings 
        # agg_debt_st.swifter.apply(lambda row1: fuzzy_comparison(fn0, ln0, row1['to whom due | first name'], row1['to whom due | last name'], state, row0, row1), axis=1) # using apply
        fuzzy_comparison_vect(index, title0, fn0, ln0, org_index0, org_file0, agg_debt_st['to whom due | title'],
                              agg_debt_st['to whom due | first name'].astype(str), agg_debt_st['to whom due | last name'].astype(str), 
                              agg_debt_st['org_index'], agg_debt_st['org_file'], state, st_driver, wait_driver) # using vectorization
        checked0.append((fn0, ln0, state))
        %store checked0

In [17]:
%%capture
# compare two strings using fuzzy string matching 
def fuzzy_comparison(index, title0, fn0, ln0, org_index0, org_file0, title1, fn1, ln1, org_index1, org_file1, 
                     state, st_driver, wait_driver):
    if (fn1, ln1, state) not in checked1:
        
        name0 = fn0 + ' ' + ln0
        name1 = fn1 + ' ' + ln1

        # use phonetic similarity (compares similar sounding names)
        meta0 = metaphone(name0.lower()) 
        meta1 = metaphone(name1.lower())
        phonetic_score = fuzz.ratio(meta0, meta1)

        # use fuzzy string similarity (compares similar spellings between names)
        fuzz_score = fuzz.ratio(name0, name1) 

        # check if phonetic score and fuzzy string score both meet threshold, both names are not the same  
        if phonetic_score > 90 and fuzz_score > 90 and name0 != name1:
            search_ancestry(index, title0, fn0, ln0, org_index0, org_file0, title1, fn1, ln1, org_index1, org_file1, name0, 
                            name1, state, st_driver, wait_driver) 
            checked1.append((fn1, ln1, state)) # record that we have checked this name
            %store checked1

In [18]:
# look up both names in ancestry's database
def search_ancestry(index, title0, fn0, ln0, org_index0, org_file0, title1, fn1, ln1, org_index1, org_file1, name0, name1, 
                    state, driver, wait):
    # loop through state urls 
    for url in records[state]:        
        try:
            # search person-0
            url0 = url + '?name=' + fn0 + '_' + ln0 + '&name_x=ps&residence=1780' + residence_urls[state] + '&residence_x=10-0-0_1-0'
            driver.get(url0) 
                
            # results were found for person0
            try:
                # use xpath to find result text
                # result0 = wait.until(presence_of_element_located((By.XPATH, '/html/body/div[3]/div/div/div/section[1]/div[1]/table/tbody/tr[2]/td[2]/span'))).text
                # use class_name to find result text
                result0 = wait.until(presence_of_element_located((By.CLASS_NAME, 'srchHit'))).text
            # no results were found; keep entries separate  
            except:
                result0 = ''
            
            # search person1
            url1 = url + '?name=' + fn1 + '_' + ln1 + '&name_x=ps&residence=1780' + residence_urls[state] + '&residence_x=10-0-0_1-0'
            driver.get(url1)
                        
            # results were found for person1
            try: 
                # use xpath to find result text
                # result1 = wait.until(presence_of_element_located((By.XPATH, '/html/body/div[3]/div/div/div/section[1]/div[1]/table/tbody/tr[2]/td[2]/span'))).text
                # use class_name to find result text
                result1 = wait.until(presence_of_element_located((By.CLASS_NAME, 'srchHit'))).text
            # no results were found; keep entries separate
            except:
                result1 = ''
            
            '''
            compare results:
            if both results are empty, do not add to fixes dict 
            if both results are different, do not add to fixes dict
            if both results are the same, add to fixes dict
                find correct name
                if name0 = result0 and result1 : {name1 : name0}
                if name1 = result1 and result0 : {name0 : name0} 
            '''
            print('---------------------------+')
            if result0 == result1 and result0 != '' and result1 != '':
                if name0 == result0 and name0 == result1: # name0 must be the correct version of the name 
                    # fixes[state][(fn1, ln1, name1)] = (fn0, ln0, name0) # convert name1 to name0  
                    # record change
                    ancestry_name_changes.append([title1, title1, fn1, ln1, fn0, ln0, 6, org_file1, org_index1, state])
                    %store ancestry_name_changes

                    
                elif name1 == result0 and name1 == result1: # name1 must be the correct version of the name 
                    # fixes[state][(fn0, ln0, name0)] = (fn1, ln1, name1) # convert name0 to name1
                    # record change
                    ancestry_name_changes.append([title0, title0, fn0, ln0, fn1, ln1, 6, org_file0, org_index0, state])
                    %store ancestry_name_changes

            print('Summary')
            print('name0=' + str(name0))
            print('name1=' + str(name1))
            print('fn0=' + str(fn0))
            print('ln0=' + str(ln0))
            print('fn1=' + str(fn1))
            print('ln1=' + str(ln1))
            print('url-0=' + str(url0))
            print('url-1=' + str(url1))
            print('result0=' + str(result0))
            print('result1=' + str(result1))
            print('state=' + str(state))
            print('rerun rows length=' + str(len(rerun_rows)))
            print('name changes length=' + str(len(ancestry_name_changes)))
            print('index=' + str(index))
            print('file_name0=' + str(org_file0))
            print('file_name1=' + str(org_file1))
            print('---------------------------+')
        
        # there was error trying to access ancestry's records
        except Exception as e:
            print('---------------------------+')
            print('Error')
            print(e)
            print('name0=' + str(name0))
            print('name1=' + str(name1))
            print('title1=' + str(type(title1)))
            print('title0=' + str(type(title0)))
            print('title1=' + str(title1))
            print('title0=' + str(title0))
            print('org_file1=' + str(org_file1))
            print('org_file0=' + str(org_file0))
            print('index=' + str(index))
            print('---------------------------+')
            rerun_rows.append([fn0, ln0, fn1, ln1, name0, name1, state]) 
            %store rerun_rows

In [ ]:
# record how long it takes to run ancestry search; useful information to see effectiveness of different methods 

# vectorize our functions 
compare_strings_vect = np.vectorize(compare_strings)
fuzzy_comparison_vect = np.vectorize(fuzzy_comparison)

# initialize a parallelization job; the idea is to have one core work on one state's agg debt file
ancestry_calls = [delayed(ancestry_cleaning)(agg_debt_sp.get_group(st), st) for st in agg_debt_sp.groups]
%timeit results = Parallel(n_jobs=-1, backend="threading")(ancestry_calls) 

Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=John Keilan
name1=John Keelan
fn0=John
ln0=Keilan
fn1=John
ln1=Keelan
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=John_Keilan&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=John_Keelan&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
rerun rows length=119
name changes length=2041
index=32443
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=Pierce_Certs_cleaned_2019.xlsx
---------------------------+
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=John Keilan
name1=John Keelan
fn0=John
ln0=Keilan
fn1=John
ln1=Keelan
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=John_Keilan&name_x=ps&residence=1780

Stored 'checked0' (list)
---------------------------+
Summary
name0=Nicholas Keimer
name1=Nicholas Keim
fn0=Nicholas
ln0=Keimer
fn1=Nicholas
ln1=Keim
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=Nicholas_Keimer&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=Nicholas_Keim&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=Nicholas Kemmer
result1=Nichs Keim
state=pa
rerun rows length=119
name changes length=2042
index=32444
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=loan_office_certificates_9_states.xlsx
---------------------------+
---------------------------+
Summary
name0=Benjamin Grovier
name1=Beniamin Grovier
fn0=Benjamin
ln0=Grovier
fn1=Beniamin
ln1=Grovier
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Benjamin_Grovier&name_x=ps&residence=1780_massachusetts-usa_24&residence_x=10-0-0_1-0
url-1=https://www.ancestrylib

---------------------------+
Summary
name0=Henry Kelkner
name1=Henry Kelker
fn0=Henry
ln0=Kelkner
fn1=Henry
ln1=Kelker
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=Henry_Kelkner&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=Henry_Kelker&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
rerun rows length=119
name changes length=2042
index=32461
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=liquidated_debt_certificates_PA_stelle.xlsx
---------------------------+
Stored 'checked1' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Simeon Guilford
name1=Simeon Guilliord
fn0=Simeon
ln0=Guilford
fn1=Simeon
ln1=Guilliord
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Simeon_Guilford&name_x=ps&residence=1780_massachusetts-usa_24&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.co

Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Francis Keller
name1=Francis Kellar
fn0=Francis
ln0=Keller
fn1=Francis
ln1=Kellar
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=Francis_Keller&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=Francis_Kellar&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
rerun rows length=119
name changes length=2042
index=32469
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=Pierce_Certs_cleaned_2019.xlsx
---------------------------+
---------------------------+
Summary
name0=Roase Peter
name1=Rose Peter
fn0=Roase
ln0=Peter
fn1=Rose
ln1=Peter
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Roase_Peter&name_x=ps&residence=1780_new+york-usa_35&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/5058/?name=R

---------------------------+
Stored 'ancestry_name_changes' (list)
Summary
name0=Thomas Kelley
name1=Thomas Kelly
fn0=Thomas
ln0=Kelley
fn1=Thomas
ln1=Kelly
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=Thomas_Kelley&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=Thomas_Kelly&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=Thomas Kelly
result1=Thomas Kelly
state=pa
rerun rows length=119
name changes length=2043
index=32486
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=Pierce_Certs_cleaned_2019.xlsx
---------------------------+
Stored 'checked1' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Jesse Guistin
name1=Jesse Guisten
fn0=Jesse
ln0=Guistin
fn1=Jesse
ln1=Guisten
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Jesse_Guistin&name_x=ps&residence=1780_massachusetts-usa_24&residence_x=10-0-0_1-0

...

Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=John M'Collum
name1=John Collum
fn0=John
ln0=M'Collum
fn1=John
ln1=Collum
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=John_M'Collum&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=John_Collum&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
rerun rows length=120
name changes length=2071
index=36246
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=liquidated_debt_certificates_PA_stelle.xlsx
---------------------------+
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Joseph Hutchins
name1=Joseph Hutchinson
fn0=Joseph
ln0=Hutchins
fn1=Joseph
ln1=Hutchinson
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Joseph_Hutchins&name_x=ps&residence=1780_massachusetts-usa_24&residence_

Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Joseph Hutchinson
name1=Joseph Hutchins
fn0=Joseph
ln0=Hutchinson
fn1=Joseph
ln1=Hutchins
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Joseph_Hutchinson&name_x=ps&residence=1780_massachusetts-usa_24&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/5058/?name=Joseph_Hutchins&name_x=ps&residence=1780_massachusetts-usa_24&residence_x=10-0-0_1-0
result0=
result1=
state=ma
rerun rows length=120
name changes length=2071
index=31441
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=Pierce_Certs_cleaned_2019.xlsx
---------------------------+
Stored 'checked1' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=John M'Cowan
name1=John M'Gowan
fn0=John
ln0=M'Cowan
fn1=John
ln1=M

Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Nicholas C. Bogart
name1=Nicholas Bogart
fn0=Nicholas C.
ln0=Bogart
fn1=Nicholas
ln1=Bogart
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Nicholas C._Bogart&name_x=ps&residence=1780_new+york-usa_35&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/5058/?name=Nicholas_Bogart&name_x=ps&residence=1780_new+york-usa_35&residence_x=10-0-0_1-0
result0=Nicholas C Bogert
result1=Nicholas Van De Bogert
state=ny
rerun rows length=120
name changes length=2072
index=124618
file_name0=loan_office_certificates_9_states.xlsx
file_name1=liquidated_debt_certificates_NY.xlsx
---------------------------+
Stored 'checked1' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=James M'Culloch
name1=James McCulloch
fn0=James
ln0=M'Culloch
fn1=James
ln1=McCulloch
url-0=https://www.ancestrylibrary.

---------------------------+
Summary
name0=Dirck Ten Broeck
name1=Dirk Ten Broeck
fn0=Dirck
ln0=Ten Broeck
fn1=Dirk
ln1=Ten Broeck
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Dirck_Ten Broeck&name_x=ps&residence=1780_new+york-usa_35&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/5058/?name=Dirk_Ten Broeck&name_x=ps&residence=1780_new+york-usa_35&residence_x=10-0-0_1-0
result0=
result1=Dirck Ten Brock 
state=ny
rerun rows length=120
name changes length=2074
index=124645
file_name0=loan_office_certificates_9_states.xlsx
file_name1=loan_office_certificates_9_states.xlsx
---------------------------+
Stored 'checked1' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
Stored 'checked0' (list)
---------------------------+
Summary
name0=Joseph Ingalls
name1=Joseph Ingals
fn0=Joseph
ln0=Ingalls
fn1=Joseph
ln1=Ingals
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Joseph_Ingalls&name_x=ps&residence=1780_massachus

---------------------------+
Summary
name0=Forrest M'Cutcheon
name1=Forrest M'Cutchion
fn0=Forrest
ln0=M'Cutcheon
fn1=Forrest
ln1=M'Cutchion
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=Forrest_M'Cutcheon&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=Forrest_M'Cutchion&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
rerun rows length=120
name changes length=2076
index=36349
file_name0=Pierce_Certs_cleaned_2019.xlsx
file_name1=Pierce_Certs_cleaned_2019.xlsx
---------------------------+
---------------------------+
Summary
name0=Samuel Ingersol
name1=Samuel Ingersole
fn0=Samuel
ln0=Ingersol
fn1=Samuel
ln1=Ingersole
url-0=https://www.ancestrylibrary.com/search/collections/5058/?name=Samuel_Ingersol&name_x=ps&residence=1780_massachusetts-usa_24&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/5058/?nam